In [ ]:
from repo.uptechStar.module.close_loop_controller import CloseLoopController
from repo.uptechStar.constant import MOTOR_DIRS, MOTOR_IDS

controller = CloseLoopController(motor_ids=MOTOR_IDS,
                                 motor_dirs=MOTOR_DIRS,
                                 port='COM1',
                                 debug=False)

controller.open_userInput_channel()





In [ ]:
from repo.uptechStar.module.actions import ActionFrame, ActionPlayer

#动作帧内定义了一个最小的动作单元，可以通过多个动作帧的嵌套组合来实现复杂的链式动作
#动作帧内置了持久化缓存功能，会将生成过的动作帧加入缓存目录，下次相同的生成请求出现时会直接返回缓存对象


#创建动作帧播放器对象
player = ActionPlayer()

#向plyer播放队列中注入一个动作帧，并马上播放，这里未向动作帧对象初始化传递参数，代表停止动作帧
player.append(action=ActionFrame(),
              play_now=True)

#生成一个动作帧列表，这里实现的是一个以1000速度前进300ms后停止运动，无动作中断，动作过程中不挂起串口通信
tape = [ActionFrame(action_speed=(1000, 1000, 1000, 1000),
                    action_duration=300,
                    breaker_func=None,
                    break_action=None,
                    hang_time=0.),
        ActionFrame()]

#注入动作帧列表到播放队列，并马上播放
player.extend(actions=tape,
              play_now=True)


In [ ]:
from repo.uptechStar.module.actions import new_ActionFrame, ActionPlayer

"""
接下来使用动作帧工厂函数来生成动作帧
动作帧工厂生成相比与直接使用实例生成具有一些额外功能
1. 支持使用multiplier，动作速度和动作时间均可接受multiplier
2. 同时也具有持久化缓存的功能，与动作帧内置的持久化缓存逻辑是一致的，用来减少相同实例的创建时间
3. 支持3种不同方式的动作速度输入分别是【全轮控制】【左右轮组分别控制】【四轮分别控制】
4. 支持自动计算串口挂起时间,注意，这里不能手动直接给出具体的挂起时间，只能决定是否使用挂起，当未指定是否使用该功能时，会在未指定breaker时激活自动挂起的功能
"""

#创建动作帧播放器对象
player = ActionPlayer()

#动作帧工厂默认返回停止动作帧，这里会使得车辆停止
player.append(action=new_ActionFrame(),
              play_now=True)

#动作帧工厂的创建方式和直接使用类创建大体是类似的

#1.使用全轮控制的模式，直接给定一个整型便代表所有轮子的速度，轮子的速度为正会让车前进，为负会让车后退
player.append(action=new_ActionFrame(action_speed=2000,
                                     action_speed_multiplier=1.2,
                                     action_duration=500,
                                     action_duration_multiplier=1.2,
                                     breaker_func=None,
                                     break_action=None,
                                     hang_during_action=True),
              play_now=True)

#2.使用左右轮组分别控制，直接给定一个Tuple[int, int]便分别按次序代表左边轮组，右边轮组的速度，同理轮子的速度为正会让车前进，为负会让车后退
player.append(action=new_ActionFrame(action_speed=(2000, 2800),
                                     action_speed_multiplier=1.2,
                                     action_duration=500,
                                     action_duration_multiplier=1.2,
                                     breaker_func=None,
                                     break_action=None,
                                     hang_during_action=True),
              play_now=True)

#3.四轮分别控制，直接给定一个Tuple[int, int, int, int]便分别按次序代表fl,rl,rr,fr轮子的速度，同理轮子的速度为正会让车前进，为负会让车后退
player.append(action=new_ActionFrame(action_speed=(2000, 2800, 5000, 6000),
                                     action_speed_multiplier=1.2,
                                     action_duration=500,
                                     action_duration_multiplier=1.2,
                                     breaker_func=None,
                                     break_action=None,
                                     hang_during_action=True),
              play_now=True)

In [ ]:
import random
import time
from matplotlib import pyplot as plt

from repo.uptechStar.module.db_tools import persistent_cache
from repo.uptechStar.module.algrithm_tools import bake_to_cache, performance_evaluate


#测试缓存的性能影响
@persistent_cache('cache')
def my_function(x):
    print("Calculating...")
    time.sleep(0.01)
    return x + 1


def learning_curve(function=my_function, multiplier: int = 2, laps: int = 20):
    op_durations_list = []

    for _ in range(laps):
        start_time = time.time()
        for i in range(256):
            # 第一次调用会计算并缓存结果
            print(function(random.randint(0, 256 * multiplier)))
        op_durations_list.append(time.time() - start_time)
    plt.plot(range(laps), op_durations_list)
    plt.show()


# learning_curve(multiplier=10)
# bake_to_cache()
performance_evaluate()